In [137]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy
import networkx as nx

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.wheel import wheel
from utils.calcium import calcium
from utils.network import network
from utils.animal_database import animal_database


Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/cat/code/donatolab/manifolds/utils/animal_database/animal_database.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [121]:
################################################
############### NETWORK ANALYSIS ###############
################################################
root_dir = '/media/cat/4TB/donato/'

#

animal_ids = [
              'DON-003165',
              'DON-003343',
              'DON-006084',
              'DON-006085',
]

sessions = [
           'DON-003165_20210211',
            'DON-003343_20210213',
            '20210519',
            '20210519'
    
]

clrs = ['black','blue','red','green']

fig = plt.figure(figsize=(12, 12))
axgrid = fig.add_gridspec(5, 4)
ax0 = fig.add_subplot(axgrid[0:3, :])
ax1 = fig.add_subplot(axgrid[3:, :2])
ax2 = fig.add_subplot(axgrid[3:, 2:])


ctr=0
corr_thresh = 0.75
for animal_id, session in zip(animal_ids, sessions):
        # 
    fname = os.path.join(root_dir,
                         animal_id, 
                         session,
                         'suite2p/plane0/cell_correlations.npy')

    # 
    cm = network.load_correlation_matrix(fname, corr_thresh)  
    print (cm.shape)
    if True:
        
        n_cells = 300
        #idx = np.random.choice(np.arange(cm.shape[0]),n_cells, replace=False)
        idx = np.arange(n_cells)
        #print (idx)
        cm = cm[idx][:,idx]

    np.fill_diagonal(cm, 0)

    #
    G = network.generate_graph_from_connected_nodes(cm)


    idx = nx.number_of_isolates(G)
    print ("# of isolates: ", idx, " , percent: ", idx/n_cells)


    # 
    degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
    # dmax = max(degree_sequence)

    #fig = plt.figure("Degree of a random graph", figsize=(8, 8))
    # Create a gridspec for adding subplots of different sizes

    Gcc = G.subgraph(sorted(nx.connected_components(G), key=len, reverse=True)[0])
    pos = nx.spring_layout(Gcc, 
                           seed=10396953)

    # 
    nx.draw_networkx_nodes(Gcc, 
                           pos, 
                           ax=ax0, 
                           node_size=20
                          )
    nx.draw_networkx_edges(Gcc, 
                           pos, 
                           ax=ax0, 
                           alpha=0.4)

    # 
    ax0.set_title("Network graph of "+animal_id + " "+session)
    ax0.set_axis_off()

    # 
    ax1.plot(degree_sequence, "b-", marker="o", c=clrs[ctr],
            label=animal_id+' '+session)
    ax1.set_title("Degree Rank Plot")
    ax1.set_ylabel("Degree")
    ax1.set_xlabel("Rank")


    #ax2.bar(*np.unique(degree_sequence, return_counts=True))
    ax2.plot(*np.unique(degree_sequence, return_counts=True),
            c=clrs[ctr])
    ax2.set_title("Degree histogram")
    ax2.set_xlabel("Degree")
    ax2.set_ylabel("# of Nodes")
    ax2.set_ylim(0,30)
    ax2.set_xlim(0,50)
    #plt.semilogy()
    #plt.semilogx()
    ctr+=1

ax1.legend()
fig.tight_layout()
plt.show()



(657, 657)
# of isolates:  258  , percent:  0.86


/tmp/ipykernel_458916/3341744120.py:89: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "b-" (-> color='b'). The keyword argument will take precedence.
  ax1.plot(degree_sequence, "b-", marker="o", c=clrs[ctr],


(840, 840)
# of isolates:  258  , percent:  0.86


/tmp/ipykernel_458916/3341744120.py:89: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "b-" (-> color='b'). The keyword argument will take precedence.
  ax1.plot(degree_sequence, "b-", marker="o", c=clrs[ctr],


(531, 531)
# of isolates:  265  , percent:  0.8833333333333333


/tmp/ipykernel_458916/3341744120.py:89: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "b-" (-> color='b'). The keyword argument will take precedence.
  ax1.plot(degree_sequence, "b-", marker="o", c=clrs[ctr],


(1232, 1232)
# of isolates:  276  , percent:  0.92


/tmp/ipykernel_458916/3341744120.py:89: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "b-" (-> color='b'). The keyword argument will take precedence.
  ax1.plot(degree_sequence, "b-", marker="o", c=clrs[ctr],


In [126]:
################################################
############### NETWORK ANALYSIS ###############
################################################
root_dir = '/media/cat/4TB/donato/'

#

animal_ids = [
              'DON-003165',
              'DON-003343',
              'DON-006084',
              'DON-006085',
]

sessions = [
           'DON-003165_20210211',
            'DON-003343_20210213',
            '20210519',
            '20210519'
    
]

clrs = ['black','blue','red','green']

#
corrs = np.arange(0.3,1.0,0.05)
print ("# corrs: ", corrs.shape)
ratios = np.zeros((14,4),'float32')
ctrc =0
for corr_thresh in corrs:
    ctra =0
    for animal_id, session in zip(animal_ids, sessions):
            # 
        fname = os.path.join(root_dir,
                             animal_id, 
                             session,
                             'suite2p/plane0/cell_correlations.npy')

        # 
        cm = network.load_correlation_matrix(fname, corr_thresh)  
        print (cm.shape)
        if True:

            n_cells = 300
            #idx = np.random.choice(np.arange(cm.shape[0]),n_cells, replace=False)
            idx = np.arange(n_cells)
            #print (idx)
            cm = cm[idx][:,idx]

        np.fill_diagonal(cm, 0)

        #
        G = network.generate_graph_from_connected_nodes(cm)


        idx = nx.number_of_isolates(G)
        print ("# of isolates: ", idx, " , percent: ", idx/n_cells)
        
        ratios[ctrc ,ctra ]= idx/n_cells
        
        ctra+=1
    ctrc+=1


# corrs:  (14,)
(657, 657)
# of isolates:  5  , percent:  0.016666666666666666
(840, 840)
# of isolates:  10  , percent:  0.03333333333333333
(531, 531)
# of isolates:  13  , percent:  0.043333333333333335
(1232, 1232)
# of isolates:  3  , percent:  0.01
(657, 657)
# of isolates:  9  , percent:  0.03
(840, 840)
# of isolates:  25  , percent:  0.08333333333333333
(531, 531)
# of isolates:  20  , percent:  0.06666666666666667
(1232, 1232)
# of isolates:  11  , percent:  0.03666666666666667
(657, 657)
# of isolates:  24  , percent:  0.08
(840, 840)
# of isolates:  47  , percent:  0.15666666666666668
(531, 531)
# of isolates:  43  , percent:  0.14333333333333334
(1232, 1232)
# of isolates:  31  , percent:  0.10333333333333333
(657, 657)
# of isolates:  53  , percent:  0.17666666666666667
(840, 840)
# of isolates:  72  , percent:  0.24
(531, 531)
# of isolates:  64  , percent:  0.21333333333333335
(1232, 1232)
# of isolates:  56  , percent:  0.18666666666666668
(657, 657)
# of isolates:  88

In [134]:
fig=plt.figure()
for k in range(4):
    plt.plot(corrs, ratios[:,k], c=clrs[k], label=animal_ids[k])
plt.xlabel("Correlation threshold for graph")
plt.ylabel("% of cells that are isolates (0-degree)")
plt.legend()
plt.show()

In [153]:
##############################################
root_dir = '/media/cat/4TB/donato/'

animal_ids = [
              'DON-003165',
              'DON-003343',
              'DON-006084',
              'DON-006085',
]
ctra=0
fig=plt.figure()
for animal_id in animal_ids: 
    ad = animal_database.AnimalDatabase()
    ad.load_sessions(animal_id)
    sessions = ad.sessions
    
    # 
    xs = []
    ys = []
    for session in sessions: 
        fname = os.path.join(root_dir,
                             animal_id, 
                             session,
                             'suite2p/plane0/cell_correlations.npy')

        # 
        corr_thresh = 0.3
        cm = network.load_correlation_matrix(fname, corr_thresh)  
        print (cm.shape)
        try:
            if True:
                n_cells = 300
                #idx = np.random.choice(np.arange(cm.shape[0]),n_cells, replace=False)
                idx = np.arange(n_cells)
                #print (idx)
                cm = cm[idx][:,idx]

            np.fill_diagonal(cm, 0)

            #
            G = network.generate_graph_from_connected_nodes(cm)

            # 
            degrees = []
            for k in G.degree():
                degrees.append(k[1])
                
            y = np.histogram(degrees, bins=np.arange(0,100,2))
            #print (y[1])
            
            ys.append(y[0])
        except:
            pass

    ys = np.array(ys)
    print ("Ys: ", ys.shape)
    median = np.median(ys,axis=0)
    plt.plot(y[1][:-1],median, label=animal_id,c=clrs[ctra])
    ctra+=1
    
plt.legend()
plt.xlabel("# of neurons")
plt.ylabel("cell degree")
plt.title("Average network degree over all sessions (8-10 session)")
plt.show()


(657, 657)
(354, 354)
(337, 337)
(389, 389)
(399, 399)
(399, 399)
(305, 305)
(301, 301)
(321, 321)
(305, 305)
Ys:  (10, 49)
(840, 840)
(842, 842)
(1084, 1084)
(1023, 1023)
(778, 778)
(542, 542)
(840, 840)
(882, 882)
(605, 605)
(519, 519)
Ys:  (10, 49)
(531, 531)
(584, 584)
(470, 470)
(503, 503)
(545, 545)
(291, 291)
(462, 462)
(268, 268)
(480, 480)
(393, 393)
Ys:  (8, 49)
(1232, 1232)
(1115, 1115)
(478, 478)
(1646, 1646)
(1182, 1182)
(1181, 1181)
(1128, 1128)
(1042, 1042)
(1028, 1028)
(676, 676)
Ys:  (10, 49)


In [7]:
################################################
########## LONGITUDINAL ANALYSIS ###############
################################################

root_dir = '/media/cat/4TB/donato/'
animal_ids = [
    'DON-006084',
    'DON-006085',
    #'DON-006087',
    'DON-003343',
    'DON-003165',
    #'DON-002865'
]

# 
cmap = plt.get_cmap("viridis", 10)
fig=plt.figure()

percentages = []
ave_degrees = []
mean_degree = []
mean_ncells = []
mean_nconnected_cells = []

corr_thresh = 0.3   # min correlation for nodes to become connected
for ctra, animal_id in enumerate(animal_ids):
    print ("animal_id: ", animal_id)
    
    #
    ax=plt.subplot(2,3,ctra+1)
    
    #
    sessions = network.get_animal_sessions(animal_id)
    res = []
    mean_degree.append([])
    temp_ncells = []
    for ctr,session in enumerate(sessions):
        fname = os.path.join(root_dir,
                            animal_id, 
                            session,
                            'suite2p/plane0/cell_correlations.npy')

        # 
        cm = network.load_correlation_matrix(fname, corr_thresh)
        temp_ncells.append(cm.shape[0])
        
        #
        G = network.generate_graph(cm)
        x, y = network.get_degree_distribution(G)
        
        plt.plot(x,y,
                color=cmap(ctr))
        res.append(y)
        
        # 
        mean_degree[ctra].append(np.median(y))
        
    mean_ncells.append(np.median(temp_n_cells))
    
    # 
    y = np.median(np.vstack(res),axis=0)
    plt.plot(x,y,
                color='red',
            linewidth=3)
    ave_degrees.append(y)

   
    #
    plt.title(animal_id)
    if ctra%3==0:
        plt.ylabel("# of neurons")
    if ctr//3==1:
        plt.xlabel("degree")
    plt.xlim(1,200)
    plt.ylim(1,700)
    #
    plt.semilogx()
    plt.semilogy()
    
    ############################
    ax2 = ax.twinx()
    # 
    cumsum = np.cumsum(y)
    cumsum = cumsum/np.max(cumsum)
    percentages.append(cumsum)
    plt.plot(x,cumsum,
                color='black',
            linewidth=3)
    ax2.set_yticks(np.arange(0,1.1,.1))
    plt.ylim(0,1)
    ax2.set_ylabel("Percentage of neurons")
    #ax2.semilogy()
    
plt.show()

animal_id:  DON-006084
animal_id:  DON-006085
animal_id:  DON-006087
animal_id:  DON-003343
animal_id:  DON-003165
animal_id:  DON-002865


In [8]:
################################################
############### NETWORK ANALYSIS ###############
################################################

clrs = ['red','blue']
fig=plt.figure()
ax=plt.subplot(1,2,1)
for k in range(len(percentages)):
    plt.plot(percentages[k],
             color = clrs[k//3])
plt.ylabel("Cumulative % of cells")
plt.xlabel("Network degree")
#plt.semilogy()
#plt.semilogx()
#
ax=plt.subplot(1,2,2)
for k in range(len(percentages)):
    plt.plot(ave_degrees[k],
             color = clrs[k//3])
plt.ylabel("# of cells")
plt.xlabel("Network degree")

#plt.semilogy()
#plt.semilogx()

plt.show()





In [34]:
#############################################
######### LOAD SUITE2P BINARIZED DATA #######
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210301/'
#data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210214/suite2p/plane0/'
# data_dir = '/media/cat/4TB/donato/nathalie/plane0'
# data_dir = '/media/cat/4TB/donato/renan/renan'
#data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
data_dir = '/media/cat/4TB/donato/DON-006084/20210520/'

data_dir = '/media/cat/4TB/donato/DON-006084/20210520/'

# initialize calcium object and load suite2p data
c = calcium.Calcium()
c.verbose = True                          # outputs additional information during processing
c.recompute_binarization = False           # recomputes binarization and other processing steps; False: loads from previous saved locations if avialable
c.data_dir = data_dir
c.load_suite2p()                          # this function assumes output dirs is either in data_dir OR data_dir/suite2p/plane0/


#
c.load_binarization()
traces = c.F_upphase_bin    # c.F_upphase_bin

print ("binarized data (# cells, # time points): ", traces.shape)




  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (584, 55740)
         self.Fneu (neuropile):  (584, 55740)
         self.iscell (Suite2p cell classifier output):  (717, 2)
              of which number of good cells:  (584,)
         self.spks (deconnvoved spikes):  (584, 55740)
         self.stat (footprints structure):  (584,)
         mean std over all cells :  22.5
   todo: print binarization defaults...
binarized data (# cells, # time points):  (584, 55740)


In [35]:
#################################################
####### OPTIONAL: REMOVE DUPLICATE CELLS ########
#################################################
c.load_footprints()
c.deduplication_method = 'overlap'      # 'overlap'; 'centre_distance'
c.corr_min_distance = 8                 # min distance for centre_distance method - NOT USED HERE
c.corr_max_percent_overlap = 0.25       # max normalized cell body allowed 
c.corr_threshold = 0.3                  # max correlation allowed for high overlap; 
                                        #     note correlations computed using filtered fluorescecne not binarized
c.corr_delete_method = 'lowest_snr'     # highest_connected: removes hub neurons,keeps more cells; 
                                        # lowest_snr - removes lower SNR cells, keep less neurons
c.recompute_deduplication = False        # recompute the dedplucaiton wif new paramters are saved
c.n_cores = 8                           #  Number of cores to use for multi processing

#
c.remove_duplicate_neurons()            

#       
traces_unique = traces[c.clean_cell_ids]
print ("All cells: ", c.F.shape, "  unique cells: ", traces_unique.shape)


 ... deduplicating cells... (can take ~5 for ~1,000cells) 


8001it [00:00, 97660.61it/s]                                                                                                                              

All cells:  (584, 55740)   unique cells:  (424, 55740)


In [36]:
###############################################################
### OPTIONAL: LOAD WHEEL DATA AND QUEISCENT OR RUN PERIODS ####
###############################################################
# 
w = wheel.Wheel()
w.root_dir = os.path.join(c.data_dir.replace('suite2p/','').replace('plane0',''),    # this assumes wheel data from Steffens' matlab scripts
                          'TRD-2P')                                                   #   is saved in the root diretory of the animal
w.load_track()
w.compute_velocity()
print ("Exp time : ", w.track.velocity.times.shape[0]/w.imaging_sample_rate)

# 
w.max_velocity_quiescent = 0.001  # in metres per second
idx_quiescent = w.get_indexes_quiescent_periods()

#
w.min_velocity_running = 0.1  # in metres per second
idx_run = w.get_indexes_run_periods()



Exp time :  1858.0
   Queiscent periods (seconds):  1359.2333333333333
   Running periods (seconds):  170.1


In [60]:
#########################################################
####### RUN PCA ON ALL OR SUBSET OF TRACES ##############
#########################################################
#

# take only 200 cells; and first 10 minutes
n_cells = 200
traces_unique = traces_unique[:n_cells]
print ("Selected cells (# cells, time points): ", traces_unique.shape)

# take quiescent periods
traces_in = traces_unique[:,idx_quiescent]

#
n_sec = 1300*30
times = np.arange(n_sec)
traces_in = traces_in[:,times]

#
suffix1 = 200   # # of cells
suffix2 = 600   # # seconds
recompute=True
pca, X_pca = c.compute_PCA(traces_in, suffix1,suffix2,recompute)

# plot  variance exaplined
var_exp = pca.explained_variance_ratio_
# 
fig=plt.figure()
ax=plt.subplot(111)
plt.plot(var_exp, label='variance explained')
plt.ylabel("variance explained")
plt.xlabel("PC component")
plt.ylim(0,0.05)
plt.legend()

ax2=ax.twinx()
cumsum = np.cumsum(var_exp)
cumsum = cumsum/np.max(cumsum)
plt.plot(cumsum, c='red', label='cumulative variance explained')
#plt.semilogy()
plt.legend()


plt.show()



Selected cells (# cells, time points):  (200, 55740)


In [63]:
# 
fig=plt.figure()
ax=plt.subplot(111)
plt.plot(var_exp, label='variance explained')
plt.ylabel("variance explained")
plt.xlabel("PC component")
#plt.ylim(0,0.05)
plt.legend()
plt.semilogy()

plt.show()
